<a href="https://colab.research.google.com/github/srilamaiti/spring_2023_w266_final_project_heesuk_iris_srila/blob/main/srila/retribert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 44.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 KB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 41.2 MB/s eta 0:00:

In [ ]:
import pandas as pd
import numpy as np
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import tensorflow as tf
import os
import re
import copy

import math
from typing import Optional
import transformers
from ktrain import text

import torch
import torch.utils.checkpoint as checkpoint
from torch import nn

In [ ]:
df = pd.read_csv('train.csv')
df['full_text'] = df["full_text"].replace(re.compile(r'[\n\r\t]'), ' ', regex = True)

label_cols = df.columns[2:]
pred_col_list = ['transformed_pred_' + col for col in label_cols]

orig_df = copy.deepcopy(df)
orig_df.head()

In [ ]:
retribert_model_checkpoints = "yjernite/retribert-base-uncased"
retribert_model = transformers.BertModel.from_pretrained(retribert_model_checkpoints)
retribert_tokenizer = transformers.BertTokenizer.from_pretrained(retribert_model_checkpoints)

In [ ]:
retribert_model

In [8]:
MAX_LEN = 512

In [9]:
shuffle = np.random.permutation(np.arange(orig_df.shape[0]))
orig_df = orig_df.iloc[shuffle]
split=(0.8,0.1,0.1)
splits = np.multiply(len(orig_df), split).astype(int)
df_train, df_val, df_test = np.split(orig_df, [splits[0], splits[0] + splits[1]])

X_train, X_val, X_test = df['full_text'], df['full_text'], df['full_text']
y_train, y_val, y_test = np.array(df_train[label_cols]), np.array(df_val[label_cols]), np.array(df_test[label_cols])


In [10]:
len(df_train), len(df_val), len(df_test)

(3128, 391, 392)

In [11]:
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
%env TOKENIZERS_PARALLELISM=false

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.13.2
transformers.__version__: 4.27.4
env: TOKENIZERS_PARALLELISM=false
